# Adaptive kNN

## ACKER: Adaptive Classifier based on kNN Expected Accuracy

```
Algorithm 1 - ACKER
===============================================
Data: point p, set Range, function f, integer l
Result: Class of point p
===============================================
optimal_k = 0
max_accuracy = -1
for k in Range do
    candidate_accuracy = expected_accuracy(p, k, f, l)
    if candidate_accuracy > max_accuracy then
        optimal_k = k
        max_accuracy = candidate_accuracy
    end
end
predicted_class = kNN(p, optimal_k)
return (predicted_class)
```

```
Algorithm 2 - Expected Accuracy
===============================================
Data: point p, integer k, function f, integer l
Result: Expected Accuracy for kNN(p, k)
===============================================
Find set C_(sim,l)(p,k) of l different points p' with minimal difference to reference function w.r.t. f:
    sim(p, p') = |f(p,k) - f(p',k)|,
    C_(sim,l) = argmax(summation(sim(p,p',k)))
correctly_predicted = number of correct predictions of kNN(p',k), where p' element_of C_(sim,l)(p,k)
expected_accuracy = (correctly_predicted / l)
return expected_accuracy
```

## Data Preprocessing

In [ ]:
!unzip "datasets.zip"
!pip install geojson

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

## Read and process crimessf
df = pd.read_csv("crimessf.csv")
# Keep only the most-common top 16 categories
top16 = df['Category'].value_counts().head(16).index
df_top16 = df[df['Category'].isin(top16)].copy()
sf_features = df_top16[["X", "Y"]]
sf_targets = df_top16["Category"]
# Encode the targets
le = LabelEncoder()
sf_targets = le.fit_transform(sf_targets)

## Read and process twitter8 and twitter11
import geojson
with open("social-pulse-milano.geojson") as f:
    gj = geojson.load(f)

g_df = pd.DataFrame(gj['features'])
# Filter to only samples with non-empty entity lists
g_df = g_df[g_df['entities'].apply(lambda x: len(x) > 0)].reset_index(drop=True)
# Add X and Y columns to g_df based on geomPoint.geom Point values
g_df['X'] = g_df['geomPoint.geom'].apply(lambda x: x["coordinates"][0])
g_df['Y'] = g_df['geomPoint.geom'].apply(lambda x: x["coordinates"][1])
# Explode targets
g_df = g_df.explode('entities')

# twitter8 had 6,230 items with 7 local geographical items + "Italy" as a general noise feature
top_places8 = ["http://dbpedia.org/resource/Italy", # General noise feature, 1399
              "http://it.dbpedia.org/resource/Stadio_Giuseppe_Meazza", # San Siro Stadium, 1615 mentions
              "http://it.dbpedia.org/resource/Arcidiocesi_di_Milano", # Archdiocese of Milan, a region? 1447 mentions https://en.wikipedia.org/wiki/Archdiocese_of_Milan
              "http://it.dbpedia.org/resource/Duomo_di_Milano", # Milan Cathedral, 889 mentions
              "http://it.dbpedia.org/resource/Aeroporto_di_Milano-Linate", # Milan Linate city Airport, 704 mentions
              "http://it.dbpedia.org/resource/Stazione_di_Milano_Centrale", # Milano Centrale Railway Station, 655 mentions
              "http://it.dbpedia.org/resource/Piazza_del_Duomo_%28Milano%29", # Cathedral Square (Milan), 654 mentions
              #"http://it.dbpedia.org/resource/Enrico_Forlanini", # Pretty sure this means Linate city airport but I think authors missed it, 643 mentions
              "http://it.dbpedia.org/resource/AF_-_L%27Artigiano_in_Fiera", # "Craftsman at the Fair" trade fair, 390 mentions
              #"http://it.dbpedia.org/resource/Italia",
              ] # 7753 exploded samples, from 6248 tweets

#Only get samples that have entities in top_places8
top8_g_df = g_df[g_df['entities'].isin(top_places8)]
# Extract features
twitter8_features = top8_g_df[['X', 'Y']]
twitter8_targets = top8_g_df['entities']
# Encode targets
le = LabelEncoder()
twitter8_targets = le.fit_transform(twitter8_targets)

# twitter11 had 21,658 items with 7 local geographical items + "Italy" as a general noise feature + 3 more unnamed general features
top_places11 = ["http://dbpedia.org/resource/Italy", # General noise feature, 1399
              "http://it.dbpedia.org/resource/Stadio_Giuseppe_Meazza", # San Siro Stadium, 1615 mentions
              "http://it.dbpedia.org/resource/Arcidiocesi_di_Milano", # Archdiocese of Milan, a region? 1447 mentions https://en.wikipedia.org/wiki/Archdiocese_of_Milan
              "http://it.dbpedia.org/resource/Duomo_di_Milano", # Milan Cathedral, 889 mentions
              "http://it.dbpedia.org/resource/Aeroporto_di_Milano-Linate", # Milan Linate city Airport, 704 mentions
              "http://it.dbpedia.org/resource/Stazione_di_Milano_Centrale", # Milano Centrale Railway Station, 655 mentions
              "http://it.dbpedia.org/resource/Piazza_del_Duomo_%28Milano%29", # Cathedral Square (Milan), 654 mentions
              #"http://it.dbpedia.org/resource/Enrico_Forlanini", # Pretty sure this means Linate city airport but I think authors missed it, 643 mentions
              "http://it.dbpedia.org/resource/AF_-_L%27Artigiano_in_Fiera", # "Craftsman at the Fair" trade fair, 390 mentions
              #"http://it.dbpedia.org/resource/Italia",
              "http://it.dbpedia.org/resource/Milano",
              "http://it.dbpedia.org/resource/Associazione_Calcio_Milan",
              "http://it.dbpedia.org/resource/Football_Club_Internazionale_Milano",
              ]

#Only get samples that have entities in top_places11
top11_g_df = g_df[g_df['entities'].isin(top_places11)]
print(f"Top places {top11_g_df.shape=}")
# Extract features
top11_features = top11_g_df[['X', 'Y']]
twitter11_targets = top11_g_df['entities']
# Encode targets
le = LabelEncoder()
twitter11_targets = le.fit_transform(twitter11_targets)

## Recreating Figure 5 from the original paper

Figure 5 shows the mean accuracy and its standard deviation of standard kNN algorithm dependent on the choice of k on our datasets

In [ ]:
# Run this cell to recreate Figure 5 from the original paper (Fig. 2 in our report)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Gather all accuracy values
sf_accuracy_vs_k = dict()
twitter8_accuracy_vs_k = dict()
twitter11_accuracy_vs_k = dict()
for k in range(1, 200):
    # Make pipeline for this k
    my_pipe = make_pipeline(
        StandardScaler(),
        KNeighborsClassifier(n_neighbors=k),
    )
    # Run on crimessf
    sf_accuracy_vs_k[k] = np.mean(cross_val_score(my_pipe, sf_features, sf_targets, cv=10))
    # Run on twitter8
    twitter8_accuracy_vs_k[k] = np.mean(cross_val_score(my_pipe, twitter8_features, twitter8_targets, cv=10))
    # Run on twitter11
    twitter11_accuracy_vs_k[k] = np.mean(cross_val_score(my_pipe, top11_features, twitter11_targets, cv=10))

# Print best accuracy k value and accuracy for each set
print(f"twitter8 best k: {max(twitter8_accuracy_vs_k, key=twitter8_accuracy_vs_k.get)}, acc={max(twitter8_accuracy_vs_k.values())}")
print(f"twitter11 best k: {max(twitter11_accuracy_vs_k, key=twitter11_accuracy_vs_k.get)}, acc={max(twitter11_accuracy_vs_k.values())}")
print(f"crimessf best k: {max(sf_accuracy_vs_k, key=sf_accuracy_vs_k.get)}, acc={max(sf_accuracy_vs_k.values())}")

# Create a figure with three subplots
fig, axes = plt.subplots(1, 3, figsize=(12, 10.0/3.0), sharex=True, sharey=False)

# Plot twitter8_accuracy_vs_k
axes[0].plot(list(twitter8_accuracy_vs_k.keys()), list(twitter8_accuracy_vs_k.values()))
axes[0].set_title("twitter8", fontweight='bold')
axes[0].set_xlabel("k")
axes[0].set_ylabel("Accuracy")
axes[0].grid(True)

# Plot twitter11_accuracy_vs_k
axes[1].plot(list(twitter11_accuracy_vs_k.keys()), list(twitter11_accuracy_vs_k.values()))
axes[1].set_title("twitter11", fontweight='bold')
axes[1].set_xlabel("k")
axes[1].set_ylabel("Accuracy")
axes[1].grid(True)

# Plot SF_accuracy_vs_k
axes[2].plot(list(sf_accuracy_vs_k.keys()), list(sf_accuracy_vs_k.values()))
axes[2].set_title("crimessf", fontweight='bold')
axes[2].set_xlabel("k")
axes[2].set_ylabel("Accuracy")
axes[2].grid(True)

plt.tight_layout()
plt.show()

fig.savefig("Figure5_all_datasets_accuracy_vs_k.pdf")

## MaxHeap (prerequisite)

In [ ]:
import heapq
import math
from typing import Callable
import numpy as np

class MaxHeap:
    """
    Max Heaps allow tracking of the n smallest values in a list because the "maximum of the smallest numbers" is always at the root.
    """
    # Initialize the max heap
    def __init__(self, data=None):
        if data is None:
            self.data = []
        else:
            self.data = [-i for i in data]
            heapq.heapify(self.data)

    # Push item onto max heap, maintaining the heap invariant
    def push(self, item):
        heapq.heappush(self.data, -item)

    # Pop the largest item off the max heap, maintaining the heap invariant
    def pop(self):
        return -heapq.heappop(self.data)

    # Pop and return the current largest value, and add the new item
    def replace(self, item):
        return heapq.heapreplace(self.data, -item)

    # Return the current largest value in the max heap
    def top(self):
        return -self.data[0]

    def size(self):
        return len(self.data)

    def __repr__(self):
        listToStr = " ".join([str(elem) for elem in self.data])
        return listToStr + "\n"


class Pair:
    """
    Override the less-than operator __lt__ to make Pair class work with max heap
    """

    def __init__(self, distance, index):
        self.distance = distance
        self.index = index

    def __lt__(self, other):
        return self.distance < other.distance

    def __gt__(self, other):
        return self.distance > other.distance

    def __neg__(self):
        return Pair(-self.distance, self.index)

    def __repr__(self):
        return f"({self.distance}, {self.index})"

## B+ Tree Implementation (prerequisite)

In [ ]:
# Source: https://gist.github.com/savarin/69acd246302567395f65ad6b97ee503d
# Adapted to have inter-leaf pointers for fast retrieval of most-similar items
"""Simple implementation of a B+ tree, a self-balancing tree data structure that (1) maintains sort
data order and (2) allows insertions and access in logarithmic time.
"""
class Node(object):
    """Base node object.
    Each node stores keys and values. Keys are not unique to each value, and as such values are
    stored as a list under each key.
    Attributes:
        order (int): The maximum number of keys each node can hold.
    """
    def __init__(self, order):
        """Child nodes can be converted into parent nodes by setting self.leaf = False. Parent nodes
        simply act as a medium to traverse the tree."""
        self.order = order
        self.keys = []
        self.values = []
        self.leaf = True
        self.next = None
        self.prev = None

    def add(self, key, value):
        """Adds a key-value pair to the node."""
        # If the node is empty, simply insert the key-value pair.
        if not self.keys:
            self.keys.append(key)
            self.values.append([value])
            return None

        for i, item in enumerate(self.keys):
            # If new key matches existing key, add to list of values.
            if key == item:
                self.values[i].append(value)
                break

            # If new key is smaller than existing key, insert new key to the left of existing key.
            elif key < item:
                self.keys = self.keys[:i] + [key] + self.keys[i:]
                self.values = self.values[:i] + [[value]] + self.values[i:]
                break

            # If new key is larger than all existing keys, insert new key to the right of all
            # existing keys.
            elif i + 1 == len(self.keys):
                self.keys.append(key)
                self.values.append([value])

    def split(self):
        """Splits the node into two and stores them as child nodes."""
        left = Node(self.order)
        right = Node(self.order)
        mid = self.order // 2

        # Fix incoming pointers
        if self.prev:
            self.prev.next = left
        if self.next:
            self.next.prev = right

        left.keys = self.keys[:mid]
        left.values = self.values[:mid]
        left.next = right
        left.prev = self.prev

        right.keys = self.keys[mid:]
        right.values = self.values[mid:]
        right.next = self.next
        right.prev = left

        # When the node is split, set the parent key to the left-most key of the right child node.
        self.keys = [right.keys[0]]
        self.values = [left, right]
        self.leaf = False

    def is_full(self):
        """Returns True if the node is full."""
        return len(self.keys) == self.order

    def show(self, counter=0):
        """Prints the keys at each level."""
        print(counter, str(self.keys))

        # Recursively print the key of child nodes (if these exist).
        if not self.leaf:
            for item in self.values:
                item.show(counter + 1)

class BPlusTree(object):
    """B+ tree object, consisting of nodes.

    Nodes will automatically be split into two once it is full. When a split occurs, a key will
    'float' upwards and be inserted into the parent node to act as a pivot.

    Attributes:
        order (int): The maximum number of keys each node can hold.
    """
    def __init__(self, order=8):
        self.root = Node(order)

    def _find(self, node, key):
        """ For a given node and key, returns the index where the key should be inserted and the
        list of values at that index."""
        for i, item in enumerate(node.keys):
            if key < item:
                return node.values[i], i    # left child, index   (if not leaf, values holds [left, right] nodes)

        return node.values[i + 1], i + 1    # right child, index

    def _merge(self, parent, child, index):
        """For a parent and child node, extract a pivot from the child to be inserted into the keys
        of the parent. Insert the values from the child into the values of the parent.
        """
        parent.values.pop(index)
        pivot = child.keys[0]

        for i, item in enumerate(parent.keys):
            if pivot < item:
                parent.keys = parent.keys[:i] + [pivot] + parent.keys[i:]
                parent.values = parent.values[:i] + child.values + parent.values[i:]
                break

            elif i + 1 == len(parent.keys):
                parent.keys += [pivot]
                parent.values += child.values
                break

    def insert(self, key, value):
        """Inserts a key-value pair after traversing to a leaf node. If the leaf node is full, split
        the leaf node into two.
        """
        parent = None
        child = self.root

        # Traverse tree until leaf node is reached.
        while not child.leaf:
            parent = child
            child, index = self._find(child, key)

        child.add(key, value)

        # If the leaf node is full, split the leaf node into two.
        if child.is_full():
            child.split()

            # Once a leaf node is split, it consists of a internal node and two leaf nodes. These
            # need to be re-inserted back into the tree.
            if parent and not parent.is_full():
                self._merge(parent, child, index)

    def retrieve(self, key):
        """Returns a value for a given key, and None if the key does not exist."""
        child = self.root

        while not child.leaf:
            child, index = self._find(child, key)

        for i, item in enumerate(child.keys):
            if key == item:
                return child.values[i]

        return None

    # AV: Added this function and the "next" and "prev" pointers
    def find_nearest(self, key, k):
        """Returns up to k*2 values nearest to a given key."""
        child = self.root

        # Find leaf node where key should go
        while not child.leaf:
            child, index = self._find(child, key)

        max_item = child.keys[-1]
        for i, item in enumerate(child.keys):
            if key < item or item == max_item:
                center = child
                assert(center.leaf == True)

                lesser_keys = []
                lesser_values = []
                added_lesser = 0
                for j in range(i-1, -1, -1): # iterate backwards through node
                    num_values = len(child.values[j]) # number of samples with the same value
                    lesser_values.extend(child.values[j]) # add all indices with this value
                    lesser_keys.extend([child.keys[j]] * num_values) # insert the key for every index added
                    added_lesser += num_values
                    if added_lesser >= k:
                        break
                child = child.prev
                while added_lesser < k and child is not None:
                    while child.leaf == False: # Find closest leaf
                        child = child.values[1] # Go to right child
                    for j in range(len(child.values)-1, -1, -1): # iterate backwards through node
                        num_values = len(child.values[j]) # number of samples with the same value
                        lesser_values.extend(child.values[j]) # add all indices with this value
                        lesser_keys.extend([child.keys[j]] * num_values) # insert the key for every index added
                        added_lesser += num_values
                        if added_lesser >= k:
                            break
                    child = child.prev

                child = center # reset back to pivot

                greater_keys = []
                greater_values = []
                added_greater = 0
                for j in range(i+1, len(child.keys)):
                    num_values = len(child.values[j])
                    greater_values.extend(child.values[j])
                    greater_keys.extend([child.keys[j]] * num_values)
                    added_greater += num_values
                    if added_greater >= k:
                        break
                child = child.next
                while added_greater < k and child is not None:
                    while child.leaf == False: # Find closest leaf
                        child = child.values[0] # Go to left child
                    for j in range(len(child.values)):
                        num_values = len(child.values[j])
                        greater_values.extend(child.values[j])
                        greater_keys.extend([child.keys[j]] * num_values)
                        added_greater += num_values
                        if added_greater >= k:
                            break
                    child = child.next

                all_keys = lesser_keys + greater_keys       # Distances
                all_values = lesser_values + greater_values # Indices
                all_pairs = []
                # Build Pairs of (key, value)
                for i in range(len(all_keys)):
                    all_pairs.append(Pair(all_keys[i], all_values[i]))
                return all_pairs

        print("ERROR in find_nearest? returning no similar values")
        return []

    def show(self):
        """Prints the keys at each level."""
        self.root.show()


## The Adaptive kNN Implementation

In [95]:
from sklearn.neighbors import KNeighborsClassifier

class AdaptiveFastKNNClassifier(object):
    """
    Adaptive Fast KNN Classifier implemented from the paper.
    """

    def __init__(self, max_k=15):
        """
        Args:
            values_of_k_as_set (set): "Range" ⊆ N defines the set of candidates k.
            similarity_function (Callable) : "SimilarityFunction defines which similarity function should be used for estimation of the expected accuracy."
            number_of_similar_points_to_find (int): "l is the parameter for the expected accuracy estimation"
            dist_fn (Callable): distance function to use for either/both of ACKER and normal kNN?
                NOTE: This is just Euclidean distance
        """

        self.max_k = max_k
        self.values_of_k_as_set = {i for i in range(2, max_k)}
        self.k_to_index_map = {k: i for i, k in enumerate(self.values_of_k_as_set)}
        self.num_k = len(self.values_of_k_as_set)

        # self.similarity_scorer = self.lat_lon
        self.similarity_scorer = self.ave_dist
        # self.similarity_scorer = self.max_dist
        # self.similarity_scorer = self.max_ave_comb

        self.number_of_similar_points_to_find = 1


    def dist_fn(self, a, b):
        return np.linalg.norm(np.array(a) - np.array(b))

    def _find_nearest(self, x, k):
        """
        Find the k nearest neighbors of point x
        Returns a list of (distance, index) pairs
        """
        maxheap = MaxHeap()
        for i in range(k):
          maxheap.push(Pair(self.dist_fn(x, self.dataset_[i]), i))
        for j in range(k, self.dataset_.shape[0]):
            new_neighbor = Pair(self.dist_fn(x, self.dataset_[j]), j)
            if new_neighbor < maxheap.top():
              maxheap.replace(new_neighbor)

        dist_idx_pairs = []
        while maxheap.size():
          neighbor = maxheap.pop()
          dist_idx_pairs.append((neighbor.distance, neighbor.index))
        return dist_idx_pairs

    def lat_lon(self, p):
        return [(p[0], p[1]) for _ in range(self.num_k)]

    def ave_dist(self, p):
        max_k_nearest = self._find_nearest(p, self.max_k)
        scores = []
        for k in self.values_of_k_as_set:
            distances = [entry[0] for entry in max_k_nearest[:k]]
            scores.append(np.mean(distances))
        return scores

    def max_dist(self, p):
        max_k_nearest = self._find_nearest(p, self.max_k)
        scores = []
        for k in self.values_of_k_as_set:
            scores.append(max_k_nearest[k - 1][0])
        return scores

    def max_ave_comb(self, p):
        max_k_nearest = self._find_nearest(p, self.max_k)
        scores = []
        for k in self.values_of_k_as_set:
            distances = [entry[0] for entry in max_k_nearest[:k]]
            mean = np.mean(distances)
            scores.append((max_k_nearest[k - 1][0], mean))
        return scores

    def lat_lon_max_ave_comb(self, p):
        max_k_nearest = self._find_nearest(p, self.max_k)
        scores = []
        for k in self.values_of_k_as_set:
            distances = [entry[0] for entry in max_k_nearest[:k]]
            mean = np.mean(distances)
            scores.append((p[0], p[1], max_k_nearest[k - 1][0], mean))
        return scores

    def expected_accuracy(self, p_scores, k):
        """
        Expected Accuracy for kNN(p, k)
        """

        p_score = p_scores[self.k_to_index_map[k]]

        if self.similarity_scorer == self.lat_lon: # Can't use B+ trees, compute similarity across all existing points
            all_similarities = np.zeros(self.dataset_.shape[0])
            for i in range(self.dataset_.shape[0]):
                i_score = self.scores[i][self.k_to_index_map[k]]
                all_similarities[i] = -self.dist_fn(p_score, i_score)

            most_similar_l_points = np.argpartition(all_similarities, -self.number_of_similar_points_to_find)[-self.number_of_similar_points_to_find:]

            # Compute the expected accuracy
            self.f_kNN.n_neighbors = k
            most_similar_X = self.dataset_[most_similar_l_points]
            most_similar_y = self.labels_[most_similar_l_points]
            y_pred = self.f_kNN.predict(most_similar_X)
            correctly_predicted = (y_pred == most_similar_y).sum()

            return correctly_predicted / self.number_of_similar_points_to_find

        else: # Used B+ trees, find most-similar points using them

            # Returns Pair objects (i_score, i) of the l*2 most similar points from the given k's B+ tree
            most_similar = self.scores[self.k_to_index_map[k]].find_nearest(p_score, self.number_of_similar_points_to_find)
            #print(f"Looking for {self.number_of_similar_points_to_find}*2 most similar points, found {len(all_similarities)}:\n{all_similarities=}")
            all_similarities = np.array([Pair(-self.dist_fn(p_score, pair.distance), pair.index) for pair in most_similar])

            # Optimization: argpartition can find top-l similarities in linear time at worst
            most_similar_l_points = np.argpartition(all_similarities, -self.number_of_similar_points_to_find)[-self.number_of_similar_points_to_find:]
            most_similar_l_points = all_similarities[most_similar_l_points]

            # Compute the expected accuracy
            self.f_kNN.n_neighbors = k
            most_similar_X = [self.dataset_[pair.index] for pair in most_similar_l_points]
            most_similar_y = [self.labels_[pair.index] for pair in most_similar_l_points]
            y_pred = self.f_kNN.predict(most_similar_X)
            correctly_predicted = (y_pred == most_similar_y).sum()

            return correctly_predicted / self.number_of_similar_points_to_find


    def ACKER(self, p):
        """
        Adaptive Classifier based on kNN Expected Accuracy

        "The main idea of the ACKER algorithm is to find optimal k for
        each point (instance) and to use this k with standard kNN, (see
        Section 4.1)"

        "Given a point p, the algorithm estimates the expected
        accuracy for different possible values of k, and chooses a k that
        provides a maximum value of the expected accuracy, (see Section 4.2)."

        "...we need to find k so that exp_acc(p,k) = max{exp_acc(p,k)|k ∈ Range},
        where Range is defined as the range of possible values for k. The
        standard kNN classifier with optimal k is applied afterwards."

        Args:
            p: Point to classify
        """

        # Step 1: Find optimal_k for point
        optimal_k = 0
        max_accuracy = -1
        p_scores = self.similarity_scorer(p)
        for k in self.values_of_k_as_set:
            candidate_accuracy = self.expected_accuracy(p_scores, k)
            if candidate_accuracy > max_accuracy:
                optimal_k = k
                max_accuracy = candidate_accuracy

        # Step 2: Use this optimal_k with standard kNN
        self.f_kNN.n_neighbors = optimal_k
        predicted_class = self.f_kNN.predict(np.expand_dims(p, axis=0))
        predicted_class = np.squeeze(predicted_class, axis=0)
        return predicted_class

    def fit(self, X, y):
        self.dataset_ = X.copy()
        self.labels_ = y.copy()

        # For each training sample, precompute scores for each k
        if self.similarity_scorer == self.lat_lon: # Can't use B+ trees for lat_lon
            self.scores = [[] for _ in range(self.dataset_.shape[0])]
            for i in range(self.dataset_.shape[0]):
              self.scores[i] = self.similarity_scorer(self.dataset_[i])
        else:
            self.scores = [BPlusTree(order=32) for _ in range(self.num_k)]
            for i in range(self.dataset_.shape[0]):
              scores = self.similarity_scorer(self.dataset_[i])
              for j in range(self.num_k):
                  self.scores[j].insert(scores[j], i)

        self.f_kNN = KNeighborsClassifier(n_neighbors=1) # Fit once and adjust n_neighbors as needed
        self.f_kNN.fit(self.dataset_, self.labels_)

    def predict(self, X):
        predictions = np.zeros(X.shape[0], dtype=int)
        for i in range(X.shape[0]):
            predicted_label = self.ACKER(X[i])
            predictions[i] = predicted_label

        return predictions

    def score(self, X, y):
        y_pred = self.predict(X)
        num_correct_predictions = (y_pred == y).sum()
        score = num_correct_predictions / y.shape[0]

        return score

# twitter8 experiments

In [ ]:
print("Full twitter8 lat_lon CV=10")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.lat_lon
my_knn.number_of_similar_points_to_find = 1
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Cross-validate
scores = cross_val_score(my_pipe, twitter8_features, twitter8_targets, cv=10)
print(f"{scores=}")
acc = scores.mean()
print(f"Cross-validated mean accuracy: {acc}")

Full twitter8 lat_lon
scores=array([0.70360825, 0.70231959, 0.6314433 , 0.67612903, 0.58580645,
       0.76387097, 0.68129032, 0.57419355, 0.80645161, 0.72129032])
acc=np.float64(0.6846403392085135)


In [ ]:
print("Full twitter8 ave_dist CV=10")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.ave_dist
my_knn.number_of_similar_points_to_find = 175
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Cross-validate
scores = cross_val_score(my_pipe, twitter8_features, twitter8_targets, cv=10)
print(f"{scores=}")
acc = scores.mean()
print(f"Cross-validated mean accuracy: {acc}")

Full twitter8 ave_dist CV=10
scores=array([0.70876289, 0.74613402, 0.63530928, 0.74193548, 0.70322581,
       0.78709677, 0.72129032, 0.73935484, 0.80774194, 0.72387097])
acc=np.float64(0.7314722314599268)


In [ ]:
print("Full twitter8 max_dist CV=10")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.max_dist
my_knn.number_of_similar_points_to_find = 169
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Cross-validate
scores = cross_val_score(my_pipe, twitter8_features, twitter8_targets, cv=10)
print(f"{scores=}")
acc = scores.mean()
print(f"Cross-validated mean accuracy: {acc}")

Full twitter8 max_dist CV=10
scores=array([0.70103093, 0.7242268 , 0.62113402, 0.73806452, 0.69935484,
       0.73419355, 0.71741935, 0.73548387, 0.80645161, 0.72129032])
acc=np.float64(0.719864981709345)


In [ ]:
print("Full twitter8 max_ave_comb CV=10")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.max_ave_comb
my_knn.number_of_similar_points_to_find = 113
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Cross-validate
scores = cross_val_score(my_pipe, twitter8_features, twitter8_targets, cv=10)
print(f"{scores=}")
acc = scores.mean()
print(f"Cross-validated mean accuracy: {acc}")

Full twitter8 max_ave_comb CV=10
scores=array([0.70747423, 0.7306701 , 0.63659794, 0.74193548, 0.70193548,
       0.77806452, 0.72129032, 0.74451613, 0.81290323, 0.72774194])
acc=np.float64(0.730312936481543)


In [ ]:
print("Full twitter8 NEW lat_lon_max_ave_comb CV=10")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.lat_lon_max_ave_comb
my_knn.number_of_similar_points_to_find = 113
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Cross-validate
scores = cross_val_score(my_pipe, twitter8_features, twitter8_targets, cv=10)
print(f"{scores=}")
acc = scores.mean()
print(f"{acc=}")

Full twitter8 NEW lat_lon_max_ave_comb, CV=10 l=113
scores=array([0.70360825, 0.74097938, 0.62886598, 0.73806452, 0.69935484,
       0.78193548, 0.72774194, 0.73806452, 0.81419355, 0.73419355])
acc=np.float64(0.7307001995344198)


# twitter11 experiments

In [ ]:
## First, shrink to subset of twitter11
# Grab 50% of the twitter11 dataset, stratified
less_features, _, less_targets, _ = train_test_split(
    top11_features,
    twitter11_targets,
    train_size=0.50, # percent to keep
    shuffle=True,
    stratify=twitter11_targets)
print(f"{less_features.shape=}")
print(f"{less_targets.shape=}")

# Split the data subset 90:10
X_train, X_test, y_train, y_test = train_test_split(
    less_features,
    less_targets,
    test_size=0.1,
    shuffle=True,
    stratify=less_targets)

In [ ]:
print("Half twitter11 lat_lon 10% holdout")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.lat_lon
my_knn.number_of_similar_points_to_find = 1
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

Half twitter11 lat_lon
X_test.shape=(1392, 2)
test_acc=0.5854885057471264


In [ ]:
print("Half twitter11 ave_dist 10% holdout")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.ave_dist
my_knn.number_of_similar_points_to_find = 268
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

Half twitter11 ave_dist
X_test.shape=(1392, 2)
test_acc=0.6242816091954023


In [ ]:
print("Half twitter11 max_dist 10% holdout")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.max_dist
my_knn.number_of_similar_points_to_find = 251
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

Half twitter11 max_dist
X_test.shape=(1392, 2)
test_acc=0.6228448275862069


In [ ]:
print("Half twitter11 max_ave_comb 10% holdout")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.max_ave_comb
my_knn.number_of_similar_points_to_find = 191
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

Half twitter11 max_ave_comb
X_test.shape=(1392, 2)
test_acc=0.625


In [ ]:
print("Half twitter11 NEW lat_lon_max_ave_comb 10% holdout")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.lat_lon_max_ave_comb
my_knn.number_of_similar_points_to_find = 191
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

Half twitter11 NEW lat_lon_max_ave_comb
X_test.shape=(1392, 2)
test_acc=0.6142241379310345


# crimessf experiments

In [ ]:
## First, shrink to subset of crimessf
# Grab 10% of the crimessf dataset, stratified
less_features, _, less_targets, _ = train_test_split(
    sf_features,
    sf_targets,
    train_size=0.10, # Percent to KEEP
    shuffle=True,
    stratify=sf_targets)
print(f"{less_features.shape=}")
print(f"{less_targets.shape=}")

# Split the data subset 90:10
X_train, X_test, y_train, y_test = train_test_split(
    less_features,
    less_targets,
    test_size=0.1,
    shuffle=True,
    stratify=less_targets)

In [ ]:
print("10% SF lat_lon single-holdout")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.lat_lon
my_knn.number_of_similar_points_to_find = 166
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

In [ ]:
print("10% SF ave_dist single-holdout")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.ave_dist
my_knn.number_of_similar_points_to_find = 742
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

In [ ]:
print("10% SF max_dist single-holdout")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.max_dist
my_knn.number_of_similar_points_to_find = 734
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

In [85]:
print("10% SF max_ave_comb single-holdout")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.max_ave_comb
my_knn.number_of_similar_points_to_find = 290
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

5% SF max_ave_comb single-holdout
X_test.shape=(702, 2)
test_acc=0.24216524216524216


In [98]:
print("10% SF lat_lon_max_ave_comb single-holdout l=290")
my_knn = AdaptiveFastKNNClassifier()
my_knn.similarity_scorer = my_knn.lat_lon_max_ave_comb
my_knn.number_of_similar_points_to_find = 290
my_pipe = make_pipeline(StandardScaler(), my_knn)
# Single 10% holdout
my_pipe.fit(X_train, y_train)
print(f"{X_test.shape=}")
y_pred = my_pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"{test_acc=}")

10% SF lat_lon_max_ave_comb single-holdout l=290
X_test.shape=(1403, 2)
test_acc=0.2580185317177477
